# H4221 - Rapport Machine Learning

[a compléter]

Auteurs: 
- DUBILLOT Elise
- FLANDRE Corentin
- THOMAS Colin

## Introduction

L'objectif du projet est de tester différents modèles de [a compléter]

In [13]:
print("hello-world")

## Chargement du jeu de données

In [14]:
import pandas as pd

df = pd.read_csv("kc_house_data.csv")
print(df)


### Traitement du jeu de données

[blabla à compléter]

Un traitement sur la date est à réaliser. Nous faisons le choix d'identifier uniquement le mois de la vente pour l'utilisr plus tard. Le jeu de données prend en compte des dates entre Mai 2014 et Mai 2015

In [61]:
# print(df["date"])
print(f"size: {len(df)}")
print(df["date"])
for i in range(len(df)):
    year = int(df["date"][i][0:4])
    month = int(df["date"][i][4:6])
    # day = int(df["date"][i][6:8]) # donnee non utilise pour notre traitement
    new_date = "error"
    if year==2014:
        if month==5:
            new_date = "month1"
        elif month==6:
            new_date = "month2"    
        elif month==7:
            new_date = "month3" 
        elif month==8:
            new_date = "month4" 
        elif month==9:
            new_date = "month5" 
        elif month==10:
            new_date = "month6" 
        elif month==11:
            new_date = "month7"
        elif month==12:
            new_date = "month8"
    elif year==2015:
        if month==1:
            new_date = "month9"
        elif month==2:
            new_date = "month10"    
        elif month==3:
            new_date = "month11" 
        elif month==4:
            new_date = "month12" 
        elif month==5:
            new_date = "month13" 
    df["date"][i] = new_date
print("--------------------")
# print(df["date"])
print(df)
    
    


size: 21613
1149     20150217T000000
15293    20140506T000000
465      20140523T000000
16198    20150324T000000
8274     20141105T000000
              ...       
1448     20150413T000000
4411     20140804T000000
9254     20140919T000000
3914     20140611T000000
7252     20141013T000000
Name: date, Length: 21613, dtype: object


C:\Users\colin\AppData\Local\Temp\ipykernel_21256\333452338.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



--------------------
               id     date      price  bedrooms  bathrooms  sqft_living  \
1149   3421079032  month10    75000.0         1       0.00          670   
15293    40000362   month1    78000.0         2       1.00          780   
465    8658300340   month1    80000.0         1       0.75          430   
16198  3028200080  month11    81000.0         2       1.00          730   
8274   3883800011   month7    82000.0         3       1.00          860   
...           ...      ...        ...       ...        ...          ...   
1448   8907500070  month12  5350000.0         5       5.00         8000   
4411   2470100110   month4  5570000.0         5       5.75         9200   
9254   9208900037   month5  6885000.0         6       7.75         9890   
3914   9808700762   month2  7062500.0         5       4.50        10040   
7252   6762700020   month6  7700000.0         6       8.00        12050   

       sqft_lot  floors  waterfront  view  ...  grade  sqft_above  \
1149     

## Exploration du jeu de données

In [ ]:
import matplotlib.pyplot as plt

  
dfsorted = df[["price", "bathrooms"]]
dfsorted.sort_values(by=["bathrooms"], inplace=True)

dfpbyb = dfsorted.groupby("bathrooms")["price"].mean()

dfpbyb.to_numpy()
plt.plot(dfpbyb[:])

plt.xlabel('Nombre de salles de bains')
plt.ylabel('Prix moyen (en $)')
plt.title('Prix en fonction du nombre de salles de bains')
plt.show()
  
dfsorted = df[["price", "floors"]]
dfsorted.sort_values(by=["floors"], inplace=True)

dfpbyb = dfsorted.groupby("floors")["price"].mean()

dfpbyb.to_numpy()
plt.plot(dfpbyb[:])

plt.xlabel("Nombre d'étages")
plt.ylabel('Prix moyen (en $)')
plt.title('Prix en fonction du nombre d\'étages')
plt.show()


dfsorted = df[["price", "sqft_living"]]
dfsorted.sort_values(by=["sqft_living"], inplace=True)

dfpbyb = dfsorted.groupby("sqft_living")["price"].mean()

dfpbyb.to_numpy()
plt.plot(dfpbyb[:])

plt.xlabel("Nombre de mètres carrés vivables")
plt.ylabel('Prix moyen (en $)')
plt.title('Prix en fonction du nombre de mètres carrés vivables')
plt.show()


dfsorted = df[["price", "bedrooms"]]
dfsorted.sort_values(by=["bedrooms"], inplace=True)

dfpbyb = dfsorted.groupby("bedrooms")["price"].mean()

dfpbyb.to_numpy()
plt.plot(dfpbyb[:])

plt.xlabel("Nombre de chambres")
plt.ylabel('Prix moyen (en $)')
plt.title('Prix en fonction du nombre de chambres')
plt.show()


dfsorted = df[["price", "sqft_lot"]]
dfsorted.sort_values(by=["sqft_lot"], inplace=True)

dfpbyb = dfsorted.groupby("sqft_lot")["price"].mean()

dfpbyb.to_numpy()
plt.plot(dfpbyb[:])

plt.xlabel("Taille du terrain en sqrft")
plt.ylabel('Prix moyen (en $)')
plt.title('Prix en fonction de la taille du terrain')
plt.show()

dfsorted = df[["price", "zipcode"]]
dfsorted.sort_values(by=["zipcode"], inplace=True)

dfpbyb = dfsorted.groupby("zipcode")["price"].mean()

dfpbyb.to_numpy()
plt.plot(dfpbyb[:])

plt.xlabel("ZIPCODE")
plt.ylabel('Prix moyen (en $)')
plt.title('Prix en fonction du zipcode')
plt.show()


dfsorted = df[["price", "lat"]]
dfsorted.sort_values(by=["lat"], inplace=True)

dfpbyb = dfsorted.groupby("lat")["price"].mean()

dfpbyb.to_numpy()
plt.hist(df["lat"], bins = 30)

plt.xlabel("Latitude")
plt.title('Répartition des données sur la latitude')
plt.show()

dfsorted = df[["price", "long"]]
dfsorted.sort_values(by=["long"], inplace=True)

dfpbyb = dfsorted.groupby("long")["price"].mean()

dfpbyb.to_numpy()
plt.hist(df["long"])

plt.xlabel("Longitude")
plt.title('Répartition des données sur la longitude')
plt.show()


In [ ]:
import plotly.express as px

fig = px.density_mapbox(df, lat='lat', lon='long',
                        mapbox_style="stamen-terrain")
 
fig

On remarque ici plusieurs cas aberrants, que l'on traitera dans la partie traitement des données.
Par exemple, dans le cas du nombre de chambre, on remarque une donnée qui représente une maison de 33 chambres, coûtant moins qu'une maison moyenne de 7 chambres et n'ayant que 1.25 salles de bain.
On remarque également que le prix et la taille du terrain ne semblent pas avoir un lien au delà de environ 0.05 sqft. On peut supposer que cela est du à la multiplicité de facteurs influençant le prix.

